In [ ]:
"""
This script computes descriptive statistics for motion
artifacts in the hyperscanning dataset. It searches for
data on drzeuss. It computes summary stats in line with
those reported in Jolly, Sadhukha, & Chang, 2020:
https://doi.org/10.1016/j.neuroimage.2020.117207
"""

In [59]:
import pandas as pd
import numpy as np
from __future__ import division
%matplotlib inline

In [2]:
# from platform import python_version
#
# print(python_version())

2.7.6


In [22]:
subList = pd.read_csv('/afs/dbic.dartmouth.edu/usr/wheatley/jd/hyperscanning_subject_list.csv',index_col=False)
subList

In [75]:
meanFD = np.empty([subList.shape[0],4])
medianFD = np.empty([subList.shape[0],4])
hiMoProp = np.empty([subList.shape[0],4])
maxFD = np.empty([subList.shape[0],4])

In [76]:
# set high-motion volume threshold (e.g. 0.3 mm from Jolly, Sadhukha, & Chang, 2020)
hiMoThresh = 0.3 # [mm]

for SUB in range(len(subIDs)): # for each subject...

    subID = subList.subID[SUB]
    pairNum = subList.pairNum[SUB]
    site = subList.site[SUB]
    if site == 'DBIC':
        siteFolder = 'hyperscanning_DBIC_ses2'
    else:
        siteFolder = 'hyperscanning_CBS'

    for RUN in [1,2,3,4]:

        if RUN < 3: # if storytelling task

            # housekeeping specific to storytelling tasks
            dateTag = ''
            folder = 'flash/wheatley/adamb/' + siteFolder + '/sub-' + subID + '_fmriprep/fmriprep/sub-' + subID + '/ses-pair0' + str(pairNum) + '/func/'

        else:

            # housekeeping specific to control tasks
            dateTag = '_2021'
            folder = '/afs/dbic.dartmouth.edu/usr/wheatley/jd/control_tasks/'

        # get file name
        fileName = 'sub-' + subID + '_ses-pair0' + str(pairNum) + '_task-storytelling' + str(RUN) + '_run-0' + str(RUN) + '_bold_confounds_truncated' + dateTag + '.csv'

        # load csv file
        moData = pd.read_csv(folder + fileName,index_col=False)
        print('loaded a ' + str(moData.shape[0]) + ' x ' + str(moData.shape[1]) + ' csv file for ' + subID + ', run ' + str(RUN))

        # get framewise displacement vector
        fwd = moData.FramewiseDisplacement[1:moData.shape[0]]

        # get descriptive stats for framewise displacement
        meanFD[SUB,RUN-1] = np.mean(fwd) # mean
        medianFD[SUB,RUN-1] = np.median(fwd) # median
        hiMoProp[SUB,RUN-1] = len(np.where(fwd > hiMoThresh)[0]) / len(fwd) # proportion above high motion threshold
        maxFD[SUB,RUN-1]  = np.max(fwd)

loaded a 1789 x 25 csv file for sid000007, run 1
loaded a 1785 x 22 csv file for sid000007, run 2
loaded a 508 x 22 csv file for sid000007, run 3
loaded a 482 x 19 csv file for sid000007, run 4
loaded a 1802 x 10 csv file for sid000009, run 1
loaded a 1812 x 10 csv file for sid000009, run 2
loaded a 506 x 19 csv file for sid000009, run 3
loaded a 481 x 19 csv file for sid000009, run 4
loaded a 1841 x 31 csv file for sid000560, run 1
loaded a 1810 x 29 csv file for sid000560, run 2
loaded a 533 x 19 csv file for sid000560, run 3
loaded a 480 x 19 csv file for sid000560, run 4
loaded a 1782 x 10 csv file for sid000535, run 1
loaded a 1799 x 10 csv file for sid000535, run 2
loaded a 501 x 19 csv file for sid000535, run 3
loaded a 481 x 19 csv file for sid000535, run 4
loaded a 1786 x 10 csv file for sid000102, run 1
loaded a 1784 x 10 csv file for sid000102, run 2
loaded a 561 x 19 csv file for sid000102, run 3
loaded a 486 x 19 csv file for sid000102, run 4
loaded a 1805 x 10 csv file fo

In [77]:
# preallocate 2 x 4 arrays: row 0 = means, row 1 = SD, columns = runs
meanFD_grp = np.empty([2,4])
medianFD_grp = np.empty([2,4])
hiMoProp_grp = np.empty([2,4])
maxFD_grp = np.empty([2,4])

# get means and SDs
meanFD_grp[0,:] = np.mean(meanFD,axis=0)
meanFD_grp[1,:] = np.std(meanFD,axis=0)
medianFD_grp[0,:] = np.mean(medianFD,axis=0)
medianFD_grp[1,:] = np.std(medianFD,axis=0)
hiMoProp_grp[0,:] = np.mean(hiMoProp,axis=0)
hiMoProp_grp[1,:] = np.std(hiMoProp,axis=0)
maxFD_grp[0,:] = np.mean(maxFD,axis=0)
maxFD_grp[1,:] = np.std(maxFD,axis=0)

# set table column and row names
colNames = ['storytelling 1','storytelling 2','listening','reading']
rowNames = ['mean FD (mm)','median FD (mm)','p(high motion)']

# make some temporary arrays
meanArray = np.vstack((meanFD_grp[0,:],medianFD_grp[0,:]))
meanArray = np.vstack((meanArray,hiMoProp_grp[0,:]))
sdArray = np.vstack((meanFD_grp[1,:],medianFD_grp[1,:]))
sdArray = np.vstack((sdArray,hiMoProp_grp[1,:]))

# put everything in a table
summTable = pd.DataFrame(np.empty([3,4]), columns = colNames, index = rowNames, dtype = np.str)
for ROW in range(meanArray.shape[0]):
    for COL in range(meanArray.shape[1]):

        summTable.iloc[ROW,COL] = str(np.round(meanArray[ROW,COL],3)) + ' ± ' + str(np.round(sdArray[ROW,COL],3))

summTable


,storytelling 1,storytelling 2,listening,reading
mean FD (mm),0.212 ± 0.074,0.191 ± 0.064,0.077 ± 0.019,0.172 ± 0.059
median FD (mm),0.171 ± 0.054,0.156 ± 0.048,0.07 ± 0.017,0.157 ± 0.052
p(high motion),0.2 ± 0.145,0.167 ± 0.122,0.007 ± 0.011,0.108 ± 0.13


In [58]:
print('The loony bin is closing shop, greetings',
          'from The Combine!')

('The loony bin is closing shop, greetings', 'from The Combine!')
